In [1]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv
import instaloader
from datetime import datetime
from moviepy.editor import AudioFileClip
import pandas as pd

In [2]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [3]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys(config.username)
password.clear()
password.send_keys(config.password)

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [4]:
# not_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]')))

# not_button.click()

In [5]:
# Wait up to 10 seconds for the search button to be clickable on the web page
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

# Click the search button once it becomes clickable
search_button.click()

In [6]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword (loop function will come here to loop through profiles)
keyword = "@omkarsadawarte" # this needs to be replacd with a loop to open profiles step by step
searchbox.send_keys(keyword)

In [7]:
# Check if the keyword starts with "@"
if keyword.startswith("@"):
    # Removes the "@" symbol
    keyword = keyword[1:]
    
try:
    # Wait until the element with the specified XPath and text is present and clickable
    first_result = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f'//span[text()="{keyword}"]'))
    )
    #print(first_result)
    
    # Click on the found element
    first_result.click()
    print(f"Clicked on the element for keyword: {keyword}")
except Exception as e:
    print(f"An error occurred: {e}, \nProfile: {keyword} not found")

Clicked on the element for keyword: omkarsadawarte


In [8]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(10)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further
 
    initial_height = current_height  # Update the initial height for the next iteration

In [9]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all anchor elements with href attributes
    anchors = soup.find_all('a', href=True)
    # print(anchors[9])
    # print([anchor['href'] for anchor in anchors])  # Prints only the href attribute of each <a> tag
    # print([anchor['href'] for anchor in anchors if anchor['href'].startswith(("/p/", "/reel/"))])

    # Filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([anchor['href'] for anchor in anchors if anchor['href'].startswith((f"/{keyword}/reel/",f"/{keyword}/p/"))])
    # print(post_urls[9])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after: {len(unique_post_urls)}")

before: 85, after: 32


In [10]:
for items in unique_post_urls:
    print(items)

/omkarsadawarte/p/Cf9oCBkDh2M/
/omkarsadawarte/p/B0ll9hdArwn/
/omkarsadawarte/reel/CWtUzVYlySR/
/omkarsadawarte/p/Cg6HzCeJTyE/
/omkarsadawarte/p/BhVmcT7gEjh/
/omkarsadawarte/p/BhEs-3hguVr/
/omkarsadawarte/reel/BwuFOwMAU3F/
/omkarsadawarte/p/B_ArIaMlv-r/
/omkarsadawarte/p/B0cdOIxA3i2/
/omkarsadawarte/p/BqBw4ncAURD/
/omkarsadawarte/p/BwrtcaggIij/
/omkarsadawarte/p/CejGIfcDAW9/
/omkarsadawarte/p/CERq0VQF1eB/
/omkarsadawarte/p/B7G4_uGllO6/
/omkarsadawarte/p/BhTkP33AW50/
/omkarsadawarte/reel/CWyMFnYor1w/
/omkarsadawarte/p/CN_xBKarau0/
/omkarsadawarte/p/Cdp5qRPpib8/
/omkarsadawarte/p/Bv8GqvSADl0/
/omkarsadawarte/reel/CiRfY85ql9g/
/omkarsadawarte/reel/CiY9_qaPYqD/
/omkarsadawarte/p/CMGyCGFrvQU/
/omkarsadawarte/p/CL4JCc6rHiL/
/omkarsadawarte/p/BpQuuIHgt51/
/omkarsadawarte/p/B8BPODkl5Zd/
/omkarsadawarte/p/Ch-Mq3gPE-u/
/omkarsadawarte/p/CLzetB5LMdS/
/omkarsadawarte/p/CLzeEMVr9-R/
/omkarsadawarte/p/B0Ou8qUgPqX/
/omkarsadawarte/p/BuTkiFrAlY5/
/omkarsadawarte/p/BwxBzskAnp3/
/omkarsadawarte/p/CDvoQM

In [ ]:
!pip install instaloader

In [ ]:
# CODE BLOCK TO TEST POST DOWNLOAD FOR SINGLE POST
# import instaloader

# # Initialize Instaloader
# loader = instaloader.Instaloader()

# # URL of the Instagram post
# post_url = "https://www.instagram.com/p/POST_ID/"  # Replace POST_ID with actual post ID or full URL

# # Download the post
# try:
#     loader.download_post(instaloader.Post.from_shortcode(loader.context, "POST_ID"), target="downloaded_content")
#     print("Video downloaded successfully!")
# except Exception as e:
#     print("An error occurred:", e)


In [11]:
import re

# List to store extracted post shortcodes
unique_post_shortcodes = []

# # Loop through each URL and extract the post ID
for text in unique_post_urls:
    match = re.search(r'/(?:p|reel)/([^/]+)/', text)
    if match:
        post_id = match.group(1)   # gives the shortcode
        unique_post_shortcodes.append(post_id)

# Print the result
print(unique_post_shortcodes)
print("\n Total unique post shortcodes: " + str(len(unique_post_shortcodes)))

['Cf9oCBkDh2M', 'B0ll9hdArwn', 'CWtUzVYlySR', 'Cg6HzCeJTyE', 'BhVmcT7gEjh', 'BhEs-3hguVr', 'BwuFOwMAU3F', 'B_ArIaMlv-r', 'B0cdOIxA3i2', 'BqBw4ncAURD', 'BwrtcaggIij', 'CejGIfcDAW9', 'CERq0VQF1eB', 'B7G4_uGllO6', 'BhTkP33AW50', 'CWyMFnYor1w', 'CN_xBKarau0', 'Cdp5qRPpib8', 'Bv8GqvSADl0', 'CiRfY85ql9g', 'CiY9_qaPYqD', 'CMGyCGFrvQU', 'CL4JCc6rHiL', 'BpQuuIHgt51', 'B8BPODkl5Zd', 'Ch-Mq3gPE-u', 'CLzetB5LMdS', 'CLzeEMVr9-R', 'B0Ou8qUgPqX', 'BuTkiFrAlY5', 'BwxBzskAnp3', 'CDvoQMpF0dZ']

 Total unique post shortcodes: 32


In [ ]:
# # # This is the best code (lacks replies extraction)
# # # Extract caption, hashtags, and comments

# # Initialize Instaloader
# loader = instaloader.Instaloader()

# # Login to Instagram using instaloader
# # instagram login and instaloader login operate independently
# username = config.username  # Replace with your Instagram username
# password = config.password  # Replace with your Instagram password
# loader.login(username, password)


# # Create a folder for the posts 
# target_folder = "downloaded_content"
# os.makedirs(target_folder, exist_ok=True)

# # Loop through each post ID
# for post_shortcode in unique_post_shortcodes:
#     try:
#         # Load the post
#         post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

#         # Download media content for the post (create a separate function for downloading media)
#         # create a check if the post_id was previously downloaded
#         loader.download_post(post, target=target_folder)

        

# #         if post.caption:
# #             caption = post.caption 
# #         else:
# #             caption = "No Captions"
        
# #         if post.caption_hashtags:
# #             hashtags = post.caption_hashtags
# #         else:
# #             hashtags = []
        
# #         # Handle comments as a list
# #         comments = []
# #         for comment in post.get_comments():
# #             comments.append({
# #                 "user": comment.owner.username,
# #                 "text": comment.text,
# #                 "created_at": comment.created_at_utc
# #             })
    
# #         # Define a unique metadata file for each post
# #         metadata_file_path = os.path.join(target_folder, f"metadata.txt")
        
# #         # Save caption, hashtags, and comments to a file in the target folder
# #         with open(metadata_file_path, "a", encoding="utf-8") as file:
# #             file.write (f"Post_ID: {post_shortcode}")
# #             file.write("Caption:\n")
# #             file.write(caption + "\n\n")
        
# #             file.write("Hashtags:\n")
# #             if hashtags:
# #                 file.write(" ".join(f"#{tag}" for tag in hashtags) + "\n\n")
# #             else:
# #                 file.write("  No Hashtags" + "\n\n")
        
# #             file.write("Comments:\n")
# #             if comments:
# #                 for comment in comments:
# #                     file.write(f"{comment['user']} ({comment['created_at']}): {comment['text']}\n\n")
# #             else:
# #                 file.write("  No Comments\n\n")
        
# #         print(f"Downloaded post and metadata for {post_shortcode} successfully!")
    
#     except Exception as e:
#         print(f"An error occurred with post {post_shortcode}: {e}")

In [19]:
import os
import instaloader
from datetime import datetime

# Initialize Instaloader
loader = instaloader.Instaloader()

# Login to Instagram using instaloader
username = config.username  # Replace with your Instagram username
password = config.password  # Replace with your Instagram password
loader.login(username, password)

from datetime import datetime

keyword = ['omkarsadawarte']

# Define target folder
target_folder = "downloaded_content"
# profile_folder = keyword[0]
# today_date = datetime.now().strftime('%Y-%m-%d')  # Format: YYYY-MM-DD
# target_folder = parent_folder + "\\" + str(today_date) + "\\" + str(profile_folder)
os.makedirs(target_folder, exist_ok=True)

# Example list of post shortcodes (replace with actual post shortcodes)
# unique_post_shortcodes = ['Cf9oCBkDh2M', 'B0ll9hdArwn', 'CWtUzVYlySR', 'Cg6HzCeJTyE', 'BhVmcT7gEjh', 'BhEs-3hguVr', 'BwuFOwMAU3F', 'B_ArIaMlv-r', 'B0cdOIxA3i2', 'BqBw4ncAURD', 'BwrtcaggIij', 'CejGIfcDAW9', 'CERq0VQF1eB', 'B7G4_uGllO6', 'BhTkP33AW50', 'CWyMFnYor1w', 'CN_xBKarau0', 'Cdp5qRPpib8', 'Bv8GqvSADl0', 'CiRfY85ql9g', 'CiY9_qaPYqD', 'CMGyCGFrvQU', 'CL4JCc6rHiL', 'BpQuuIHgt51', 'B8BPODkl5Zd', 'Ch-Mq3gPE-u', 'CLzetB5LMdS', 'CLzeEMVr9-R', 'B0Ou8qUgPqX', 'BuTkiFrAlY5', 'BwxBzskAnp3', 'CDvoQMpF0dZ']

# List to store paths of downloaded .mp4 files
downloaded_media = []

# Loop through each post shortcode
for post_shortcode in unique_post_shortcodes:
    try:
        # Load the post
        post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

        # Determine filename based on post type
        if post.is_video:
            filename = f"{post.owner_username}_{post_shortcode}.mp4"
            
            # Download the post
            loader.download_post(post, target=target_folder)
            
            # Find and rename the downloaded .mp4 file
            for file in os.listdir(target_folder):
                if file.endswith(".mp4"):
                    # Rename the file to our custom filename
                    old_path = os.path.join(target_folder, file)
                    new_path = os.path.join(target_folder, filename)
                    os.rename(old_path, new_path)

                    # Store the complete file path of the downloaded media
                    file_path = r"E:\work\web_scraping_101\soc_listn" +"\\"+ str(new_path)
                    downloaded_media.append(file_path)
                    print(f"Downloaded and renamed file to: {filename}")
                    break  # Exit the loop once the file is renamed

    except Exception as e:
        print(f"An error occurred with post {post_shortcode}: {e}")

# Print the list of downloaded video file paths
print("Downloaded Media Files:")
for media in downloaded_media:
    print(media)

An error occurred with post Cf9oCBkDh2M: 'Post' object has no attribute 'is_reel'
An error occurred with post B0ll9hdArwn: 'Post' object has no attribute 'is_reel'
downloaded_content﹨2024-11-15﹨omkarsadawarte﹨CWtUzVYlySR\2021-11-25_18-07-03_UTC.jpg [Goodbyes can be very painful.…] downloaded_content﹨2024-11-15﹨omkarsadawarte﹨CWtUzVYlySR\2021-11-25_18-07-03_UTC.mp4 json 
An error occurred with post Cg6HzCeJTyE: 'Post' object has no attribute 'is_reel'
An error occurred with post BhVmcT7gEjh: 'Post' object has no attribute 'is_reel'
An error occurred with post BhEs-3hguVr: 'Post' object has no attribute 'is_reel'
downloaded_content﹨2024-11-15﹨omkarsadawarte﹨BwuFOwMAU3F\2019-04-26_13-14-21_UTC.jpg [So cute 😍😍 #squirrel #nature …] downloaded_content﹨2024-11-15﹨omkarsadawarte﹨BwuFOwMAU3F\2019-04-26_13-14-21_UTC.mp4 json 
An error occurred with post B_ArIaMlv-r: 'Post' object has no attribute 'is_reel'
An error occurred with post B0cdOIxA3i2: 'Post' object has no attribute 'is_reel'
An error

In [ ]:
############# trial code(UPDATED) to extract caption, hashtag, comment, replies, likes_count, comments_count & save in a dataframe ##############

# List to store all post data (flattened for DataFrame)
post_info_list = []

# Loop through each post ID
for post_shortcode in unique_post_shortcodes:
    try:
        # Load the post
        post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

        # Handle missing captions and hashtags
        caption = post.caption if post.caption else "No Captions"
        hashtags = post.caption_hashtags if post.caption_hashtags else []

        # Get the likes and comments count
        likes_count = post.likes
        comments_count = post.comments

        # List to hold comments and replies
        comments_data = []

        # Loop through each comment in the post
        for comment in post.get_comments():
            comment_info = {
                "user": comment.owner.username,
                "text": comment.text,
                "replies": []
            }

            # Check if there are replies to the comment
            if comment.answers:  # Ensure there are replies before processing
                for reply in comment.answers:
                    reply_info = {
                        "user": reply.owner.username,
                        "text": reply.text,
                    }
                    comment_info["replies"].append(reply_info)

            comments_data.append(comment_info)

            # Check for pagination and load more comments if necessary
            # while comment.has_more_comments:
            #     comment.load_more_comments()  # Load more comments if there are more pages of comments
                
                #  processing the newly replies
            for reply in comment.answers:
                reply_info = {
                    "user": reply.owner.username,
                    "text": reply.text,
                }
                comment_info["replies"].append(reply_info)

        # Prepare post data to be added to the list
        post_info = {
            "shortcode": post_shortcode,
            "caption": caption,
            "hashtags": hashtags,
            "likes_count": likes_count,
            "comments_count": comments_count,
            "comments_data": comments_data
        }

        # Append the post_info to the list
        post_info_list.append(post_info)

    except instaloader.exceptions.InstaloaderException as e:
        print(f"Error loading post {post_shortcode}: {e}")
        continue  # Handle the exception and continue with the next post

# Now save the post_info_list to a pandas DataFrame
df = pd.DataFrame(post_info_list)


In [ ]:
#################################################### example of how to flaten the structure ####################################################

# flat_data = []

# # Flatten the comments and replies data
# for post_info in post_info_list:
#     for comment in post_info['comments_data']:
#         for reply in comment['replies']:
#             flat_data.append({
#                 "shortcode": post_info["shortcode"],
#                 "caption": post_info["caption"],
#                 "hashtags": post_info["hashtags"],
#                 "likes_count": post_info["likes_count"],
#                 "comments_count": post_info["comments_count"],
#                 "comment_user": comment["user"],
#                 "comment_text": comment["text"],
#                 "reply_user": reply["user"],
#                 "reply_text": reply["text"]
#             })

# # Create a DataFrame from the flattened data
# flat_df = pd.DataFrame(flat_data)

# # Optionally, save the flattened DataFrame to a CSV file
# flat_df.to_csv('flattened_post_data.csv', index=False)

# # Display the flattened DataFrame
# print(flat_df.head())


In [ ]:
df

In [ ]:
######################### FUNCTIONALISED CODE TO EXTRACT POST DATA (captions, comments etc) AND SAVE IN A PANDAS DATAFRAME ##############################



In [ ]:
# # trail code to extract replies from comment (OLD, includes reply extraction)
# # delete if it does not extract replies from comments

# import instaloader

# # Initialize Instaloader
# loader = instaloader.Instaloader()

# # Login to Instagram using instaloader
# username = config.username  # Replace with your Instagram username
# password = config.password  # Replace with your Instagram password
# loader.login(username, password)

# # Define the post shortcode you want to analyze
# post_shortcode = "CiY9_qaPYqD"  # Replace with the Instagram post shortcode

# # Load the post
# post = instaloader.Post.from_shortcode(loader.context, post_shortcode)
# #post = instaloader.Post.from_shortcode(loader.context, post_shortcode)

# # List to hold comments and replies
# comments_data = []

# # Loop through each comment in the post
# for comment in post.get_comments():
#     comment_info = {
#         "user": comment.owner.username,
#         "text": comment.text,
#         "created_at": comment.created_at_utc,
#         "replies": []
#     }

#     # Check if there are replies to the comment
#     for reply in comment.answers:
#         reply_info = {
#             "user": reply.owner.username,
#             "text": reply.text,
#             "created_at": reply.created_at_utc
#         }
#         comment_info["replies"].append(reply_info)
        
#     comments_data.append(comment_info)

# # Print comments and their replies
# for comment in comments_data:
#     print(f"Comment by {comment['user']} ({comment['created_at']}): {comment['text']}")
#     for reply in comment["replies"]:
#         print(f"  Reply from {reply['user']} ({reply['created_at']}): {reply['text']}")

In [ ]:
!pip install moviepy

In [35]:
########################################### CODE TO EXTRACT AUDIO FROM VIDEO ###################################################

from moviepy.editor import VideoFileClip

# Function to extract audio from a video file and save as .wav
def extract_audio_as_wav(video_file_path):
    # Set the path for the .wav audio file
    audio_file_path = os.path.splitext(video_file_path)[0] + ".wav"

    try:
        # Extract audio and save as .wav
        with VideoFileClip(video_file_path) as video_clip:
            video_clip.audio.write_audiofile(audio_file_path, codec='pcm_s16le',verbose = False, logger= None)
        
        print(f"Audio file saved as: {audio_file_path}")
        return audio_file_path
    except Exception as e:
        print(f"An error occurred: {e}, \nProfile: {keyword} not found")

# Example usage
# video_file_path = "path/to/your/downloaded_video.mp4"  # Replace with your actual video file path
video_file_path = downloaded_media
# video_file_path = r""C:\Users\OMKAR\Downloads\Indian Biscuits Fact 😂 _ Viral Comedy Video_ Funny Whatsapp Status #priyalkukreja #shorts #ytshorts.mp4"  # Replace with your actual video file path
# audio_file = extract_audio_as_wav(video_file_path)
saved_audio_files = []
for video_path in video_file_path:
    audio_file = extract_audio_as_wav(video_path)

Audio file saved as: E:\work\web_scraping_101\soc_listn\downloaded_content\omkarsadawarte_CWtUzVYlySR.wav
Audio file saved as: E:\work\web_scraping_101\soc_listn\downloaded_content\omkarsadawarte_BwuFOwMAU3F.wav


In [59]:
####################################### CODE TO SAVE THE FILE PATH OF ALL AUDIO FILES IN A LIST ####################################

import os

# Specify the target directory
directory = "E:\\work\\web_scraping_101\\soc_listn\\downloaded_content"

# Set to store unique file paths
audio_file_paths = set()

# Loop through all files in the directory
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".wav"):
            # Construct the full file path
            full_path = os.path.join(root, file)
            # Add the file path to the set (duplicates will be automatically ignored)
            audio_file_paths.add(full_path)

# Convert the set back to a list (if you need it as a list)
audio_file_paths = list(audio_file_paths)

# Print the list of file paths
print("All file paths:")
print(audio_file_paths)


All file paths:
['E:\\work\\web_scraping_101\\soc_listn\\downloaded_content\\omkarsadawarte_CWtUzVYlySR.wav', 'E:\\work\\web_scraping_101\\soc_listn\\downloaded_content\\omkarsadawarte_BwuFOwMAU3F.wav']


In [61]:
from pathlib import Path
import whisper

# Load the Whisper model
model = whisper.load_model("small")

for audio_file in audio_file_paths:
    
    # Use Pathlib to define the path
    path = Path(str(audio_file))
    
    # Check if the file exists before transcribing
    if not path.exists():
        print(f"File not found: {audio_file_path}")
    else:
        # Transcribe the audio file
        result = model.transcribe(str(audio_file))
    
        # Print detected language and transcription
        print("Detected Language:", result["language"])
        print("Transcription:", result["text"])


Detected Language: ja
Transcription: ボクシングする意味 知ることをできた君に感謝していますえっ? 僕に?祖国に帰ったら 君に教わったことを後輩に伝えていきたいそして自慢する僕 日本で強いボクサーと戦ってきたと一歩幕のうちという 素晴らしいファイターと戦ってきたとねそんなその時はチャンピオンでいてほしい日本じゃなく もっとスケールの大きな日本じゃない約束できなくていいです今はボルグボルグボルグ時間 行かなくてはもう 古くなったけど君に持っていてほしい長くいるつもりだったから一生懸命日本を覚えた祖国戻ったら二度と使うことはないでしょうだからこれ 僕の最後の日本を幕のうちさようなら僕は僕は僕は僕は僕は僕は僕は僕は僕は僕は僕は僕は僕を忘れませんボルグさんのことをボルグさんと戦ったことを絶対忘れません
Detected Language: jw
Transcription:  වවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවව වවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවවව
